In [2]:
import os

In [5]:
import pyspark
import time
sc = pyspark.SparkContext(appName="Spark_RDD")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/15 22:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
os.getcwd()

'/Users/slimkachkachi'

In [13]:
def reducer(x):
    key=x[0]
    values=x[1]
    min=key
    valuesList=[]
    listoutput=[]
    global newpaircounter
    for i in values:
        if i <min: 
            min=i
        valuesList.append(i)
    if min < key:
        listoutput.append((key,min))
        for j in valuesList:
            if min != j:
              listoutput.append((j,min))
              newpaircounter.add(1)
    return listoutput

In [24]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/facebook_combined.txt")
rdd=rdd_init.map(lambda x:(x.split(' ')))

In [27]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/HR_edges.csv")
rdd=rdd_init.map(lambda x:(x.split(','))).filter(lambda x:x[0] != 'node_1')

In [30]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/artist_edges.csv")
rdd=rdd_init.filter(lambda x :x != 'node_1,node_2').map(lambda x:(x.split(',')))

In [33]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/web-Google.txt")
rdd=rdd_init.filter(lambda x:"#" not in x).map(lambda x:x.split("\t")).map(lambda x:(int(x[0]),int(x[1])))

In [35]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/large_twitch_edges.csv")
rdd=rdd_init.filter(lambda x :x != 'numeric_id_1,numeric_id_2').map(lambda x:(x.split(','))).map(lambda x:(int(x[0]),int(x[1])))

In [38]:
rdd_init = sc.textFile("/Users/slimkachkachi/Documents/IA_Dauphine/Traitement distribué données massives/Projet/Fichiers Edges/soc-pokec-relationships.txt")
rdd=rdd_init.filter(lambda x:"#" not in x).map(lambda x:x.split("\t")).map(lambda x:(int(x[0]),int(x[1])))


In [39]:
boucle = True
t0=time.time()

while boucle == True:
 
  newpaircounter=sc.accumulator(0)
  
  #map du ccf
  Map_rdd=rdd.flatMap(lambda x:(x,(x[1],x[0])))
  
  #fonction reduce du CCF Iterate RDD fichier de chiffres
  Reduce_rdd=Map_rdd.groupByKey().flatMap(lambda x:reducer(x)) #.filter(lambda x:x!=None).flatMap(lambda x:x)
  Reduce_rdd.count() 
  
  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  rdd=Reduce_rdd.distinct().sortByKey()

  print(newpaircounter.value)
  if newpaircounter.value == 0:
    print("arret")
    boucle = False

t1=time.time()
print("c'est la fin; durée totale de :",round(t1-t0,3))
#rdd.collect()

118273064


148592320


22/10/16 00:05:21 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 955320 ms exceeds timeout 120000 ms
22/10/16 00:05:21 WARN SparkContext: Killing executors is not supported by current scheduler.


124362218


5689920


408


0
arret
c'est la fin; durée totale de : 13401.866


In [37]:
rdd.map(lambda x:(x[1],x[0])).groupByKey().mapValues(list).map(lambda x : (x[0], len(list(x[1]))+1)).collect()

[(0, 168114)]